In [33]:
import numpy as np
import cv2 as cv
import glob
import logging
import copy

# FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS

chessboardSize = (7, 4)
frameSize = (640, 480)

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

size_of_chessboard_squares_mm = 22
objp = objp * size_of_chessboard_squares_mm

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('Images/*.png')

for image in images:

    img = cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)
    # print(corners)

    # If found, add object points, image points (after refining them)
    if ret == True:

        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(1)


cv.destroyAllWindows()

############## CALIBRATION #######################################################

ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

 ############## UNDISTORTION #####################################################

img = cv.imread('Images/2.png')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))

# Undistort
dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult1.png', dst)

# Undistort with Remapping
mapx, mapy = cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult2.png', dst)

# Reprojection Error
mean_error = 0

# for i in range(len(objpoints)):
#     imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
#     error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
#     mean_error += error
#
# print( "total error: {}".format(mean_error/len(objpoints)))

imgTest = cv.imread('caliResult1.png')
gray = cv.cvtColor(imgTest, cv.COLOR_BGR2GRAY)

# Find the chess board corners
ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)
#print(corners)

# If found, add object points, image points (after refining them)
if ret == True:

    objpoints.append(objp)
    corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
    imgpoints.append(corners)

    # Draw and display the corners
    cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
    cv.imshow('img', img)
    cv.waitKey(1000)

print(corners2)


[[[ 45.90908  333.01834 ]]

 [[ 44.732777 296.5466  ]]

 [[ 43.868443 259.65988 ]]

 [[ 42.60421  223.04715 ]]

 [[ 41.91295  186.33113 ]]

 [[ 41.705578 149.46207 ]]

 [[ 41.769547 113.02235 ]]

 [[ 83.65147  331.96585 ]]

 [[ 82.73407  295.60776 ]]

 [[ 81.80902  258.76434 ]]

 [[ 80.74097  222.34503 ]]

 [[ 79.82566  185.61302 ]]

 [[ 79.63622  148.8949  ]]

 [[ 79.599625 112.54216 ]]

 [[120.92403  331.28867 ]]

 [[120.22875  294.77762 ]]

 [[119.35798  258.1793  ]]

 [[118.50692  221.58434 ]]

 [[117.68216  185.1509  ]]

 [[117.52841  148.43082 ]]

 [[117.16511  112.169624]]

 [[157.81522  330.58362 ]]

 [[157.3921   293.98044 ]]

 [[156.6153   257.34177 ]]

 [[155.88177  220.95    ]]

 [[155.32243  184.45726 ]]

 [[154.88914  147.88818 ]]

 [[154.28537  111.44098 ]]]


In [34]:
np.shape(np.squeeze(corners2))

(28, 2)

In [35]:
objp

array([[  0.,   0.,   0.],
       [ 22.,   0.,   0.],
       [ 44.,   0.,   0.],
       [ 66.,   0.,   0.],
       [ 88.,   0.,   0.],
       [110.,   0.,   0.],
       [132.,   0.,   0.],
       [  0.,  22.,   0.],
       [ 22.,  22.,   0.],
       [ 44.,  22.,   0.],
       [ 66.,  22.,   0.],
       [ 88.,  22.,   0.],
       [110.,  22.,   0.],
       [132.,  22.,   0.],
       [  0.,  44.,   0.],
       [ 22.,  44.,   0.],
       [ 44.,  44.,   0.],
       [ 66.,  44.,   0.],
       [ 88.,  44.,   0.],
       [110.,  44.,   0.],
       [132.,  44.,   0.],
       [  0.,  66.,   0.],
       [ 22.,  66.,   0.],
       [ 44.,  66.,   0.],
       [ 66.,  66.,   0.],
       [ 88.,  66.,   0.],
       [110.,  66.,   0.],
       [132.,  66.,   0.]], dtype=float32)

In [36]:
list1 = []

In [37]:
for elements in objp:
    elements = elements[:2] 
    list1.append(elements)
list1

[array([0., 0.], dtype=float32),
 array([22.,  0.], dtype=float32),
 array([44.,  0.], dtype=float32),
 array([66.,  0.], dtype=float32),
 array([88.,  0.], dtype=float32),
 array([110.,   0.], dtype=float32),
 array([132.,   0.], dtype=float32),
 array([ 0., 22.], dtype=float32),
 array([22., 22.], dtype=float32),
 array([44., 22.], dtype=float32),
 array([66., 22.], dtype=float32),
 array([88., 22.], dtype=float32),
 array([110.,  22.], dtype=float32),
 array([132.,  22.], dtype=float32),
 array([ 0., 44.], dtype=float32),
 array([22., 44.], dtype=float32),
 array([44., 44.], dtype=float32),
 array([66., 44.], dtype=float32),
 array([88., 44.], dtype=float32),
 array([110.,  44.], dtype=float32),
 array([132.,  44.], dtype=float32),
 array([ 0., 66.], dtype=float32),
 array([22., 66.], dtype=float32),
 array([44., 66.], dtype=float32),
 array([66., 66.], dtype=float32),
 array([88., 66.], dtype=float32),
 array([110.,  66.], dtype=float32),
 array([132.,  66.], dtype=float32)]

In [38]:
np.shape(list1)

(28, 2)

In [39]:
mylist = list(zip(list1, np.squeeze(corners2)))
print(mylist)

[(array([0., 0.], dtype=float32), array([ 45.90908, 333.01834], dtype=float32)), (array([22.,  0.], dtype=float32), array([ 44.732777, 296.5466  ], dtype=float32)), (array([44.,  0.], dtype=float32), array([ 43.868443, 259.65988 ], dtype=float32)), (array([66.,  0.], dtype=float32), array([ 42.60421, 223.04715], dtype=float32)), (array([88.,  0.], dtype=float32), array([ 41.91295, 186.33113], dtype=float32)), (array([110.,   0.], dtype=float32), array([ 41.705578, 149.46207 ], dtype=float32)), (array([132.,   0.], dtype=float32), array([ 41.769547, 113.02235 ], dtype=float32)), (array([ 0., 22.], dtype=float32), array([ 83.65147, 331.96585], dtype=float32)), (array([22., 22.], dtype=float32), array([ 82.73407, 295.60776], dtype=float32)), (array([44., 22.], dtype=float32), array([ 81.80902, 258.76434], dtype=float32)), (array([66., 22.], dtype=float32), array([ 80.74097, 222.34503], dtype=float32)), (array([88., 22.], dtype=float32), array([ 79.82566, 185.61302], dtype=float32)), (arra

In [40]:
print(mylist[0][0], mylist[0][1])
print(mylist[6][0], mylist[6][1])
print(mylist[27][0], mylist[27][1])
print(mylist[3][0], mylist[3][1])

[0. 0.] [ 45.90908 333.01834]
[132.   0.] [ 41.769547 113.02235 ]
[132.  66.] [154.28537 111.44098]
[66.  0.] [ 42.60421 223.04715]


In [41]:
features_mm_to_pixels_dict = {tuple(mylist[0][0]) : tuple(mylist[0][1]),
                              tuple(mylist[6][0]) : tuple(mylist[6][1]),
                              tuple(mylist[27][0]) : tuple(mylist[27][1]),
                              tuple(mylist[3][0]) : tuple(mylist[3][1])}  

In [46]:
import logging
import cv2
import numpy as np
import copy

logging.basicConfig(level=logging.DEBUG, format='%(asctime)-15s %(levelname)s \t%(message)s')

def draw_crosshair(image, center, width, color):
    cv2.line(image, (center[0] - width//2, center[1]), (center[0] + width//2, center[1]), color, 2)
    cv2.line(image, (center[0], center[1] - width//2), (center[0], center[1] + width//2), color, 2)

image_filepath = "caliResult1.png"

A = np.zeros((2 * len(features_mm_to_pixels_dict), 6), dtype=float)
b = np.zeros((2 * len(features_mm_to_pixels_dict), 1), dtype=float)
index = 0

for XY, xy in features_mm_to_pixels_dict.items():
    X = XY[0]
    Y = XY[1]
    x = xy[0]
    y = xy[1]
    A[2 * index, 0] = x
    A[2 * index, 1] = y
    A[2 * index, 2] = 1
    A[2 * index + 1, 3] = x
    A[2 * index + 1, 4] = y
    A[2 * index + 1, 5] = 1
    b[2 * index, 0] = X
    b[2 * index + 1, 0] = Y
    index += 1
# A @ x = b
x, residuals, rank, singular_values = np.linalg.lstsq(A, b, rcond=None)

pixels_to_mm_transformation_mtx = np.array([[x[0, 0], x[1, 0], x[2, 0]], [x[3, 0], x[4, 0], x[5, 0]], [0, 0, 1]])
logging.debug("main(): pixels_to_mm_transformation_mtx = \n{}".format(pixels_to_mm_transformation_mtx))

mm_to_pixels_transformation_mtx = np.linalg.inv(pixels_to_mm_transformation_mtx)

test_XY_2 = (22, 22, 1) # Nhap toa do test
test_xy_2 = mm_to_pixels_transformation_mtx @ test_XY_2
image = cv2.imread(image_filepath)
annotated_img = copy.deepcopy(image)
draw_crosshair(annotated_img, (round(test_xy_2[0]), round(test_xy_2[1])), 60, (0, 0, 255))
cv2.imwrite("./outputs/annotatedFull.png", annotated_img)

True